In [10]:
from QAOA_ansatz import create_QAOA_ansatz, create_hhl_circ
from amp_est import real_amp_est
from taylor_precomputation import construct_asin_x_inv_circuit, construct_exp_k_abs_cos_circuit, get_asin_x_inv_expression, get_cos_expression
from problem_formulation import Node, Line, Grid, UCProblem

In [11]:
import numpy as np
from math import pi
from qiskit import QuantumCircuit, transpile, QuantumRegister, ClassicalRegister
from qiskit.providers.aer import Aer
from qiskit.circuit import ParameterVector
from qiskit.circuit.library.data_preparation.state_preparation import StatePreparation

from qiskit.opflow import PauliTrotterEvolution
from qiskit.opflow import I, X, Y, Z

In [12]:
asin_x_inv_expression=[
            2.378888462559524,
            0.0,
            0.0,
            0.0,
            -1.4557694066964286,
            0.0,
            0.0,
            0.0,
            -1.8624683605952381,
            0.0,
            0.0,
            0.0,
            1.2791832861190477,
            0.0,
            0.0,
            0.0
        ]

In [13]:
node1=Node([2,2], 5, 1, 1, "gen1")
node2=Node([1,1], 1, 2, 1, "gen2")
node3=Node([-1.5,-1], 0,0,0, "load1")
node4=Node([-1,0], 0,0,0, "load2")
line1=Line(node1,node3,1,1)
line2=Line(node2,node1,1,1)
line3=Line(node4,node2,1,1)
line4=Line(node4,node3,1,1)

problem_instance=UCProblem([line1,line2,line3, line4], [node1,node2,node3,node4], 2)

In [14]:
grid=problem_instance.grid_timesteps
grid.A

array([[ 3., -1.,  0., -1.],
       [-1.,  2., -1.,  0.],
       [ 0., -1.,  2., -1.],
       [-1.,  0., -1.,  2.]])

In [15]:
grid.A_eig_bounds

(0.18644324808654034, 5.0)

In [16]:
real_powers=np.array([node.real_power for node in grid.nodes])
print(real_powers[:,0])

[ 2.   1.  -1.  -1.5]


In [17]:
gen_nodes=QuantumRegister(2,"gen_nodes")
tot_nodes=QuantumRegister(2,"tot_nodes")
state_prep_anc=QuantumRegister(1,"state_prep_anc")
hhl_phase_reg=QuantumRegister(5,"hhl_phase_reg")
hhl_anc=QuantumRegister(1,"hhl_anc")

qc=QuantumCircuit(gen_nodes,tot_nodes,state_prep_anc,hhl_phase_reg,hhl_anc)

qc.x(gen_nodes)
qc.compose(create_hhl_circ(real_powers=real_powers[:,0],
                         B=grid.A,
                         max_eigval=grid.A_eig_bounds[1],
                         C=grid.A_eig_bounds[0],
                         gen_nodes=gen_nodes,
                         tot_nodes=tot_nodes,
                         state_prep_anc=state_prep_anc,
                         hhl_phase_reg=hhl_phase_reg,
                         hhl_anc=hhl_anc, num_time_slices=5), inplace=True)
hhl_circ=[qc]

Check if HHL circuit already exists
'HHL_[ 2.   1.  -1.  -1.5]_[[ 3. -1.  0. -1.]\n [-1.  2. -1.  0.]\n [ 0. -1.  2. -1.]\n [-1.  0. -1.  2.]]_5.0_0.18644324808654034_5_5'
Constructing HHL Circuit
Initial B: [[ 3. -1.  0. -1.]
 [-1.  2. -1.  0.]
 [ 0. -1.  2. -1.]
 [-1.  0. -1.  2.]]
Final B: [[ 3.40454829 -1.13484943  0.         -1.13484943]
 [-1.13484943  2.26969886 -1.13484943  0.        ]
 [ 0.         -1.13484943  2.26969886 -1.13484943]
 [-1.13484943  0.         -1.13484943  2.26969886]]
Constructing asin(1/(mx)) circuit
Length of untranspiled HHL 513
Transpiling HHL circuit for storage
Length of transpiled HHL 53359


In [18]:
hhl_circ[0].draw()

┌───┐                          ┌───┐»
    gen_nodes_0: ──────────────────────────┤ X ├──────────────────────────┤ X ├»
                                           ├───┤                          ├───┤»
    gen_nodes_1: ──────────────────────────┤ X ├──────────────────────────┤ X ├»
                 ┌─────────────────────────┴───┴─────────────────────────┐├───┤»
    tot_nodes_0: ┤0                                                      ├┤ X ├»
                 │  State Preparation(0.69631,0.34816,-0.34816,-0.52223) │├───┤»
    tot_nodes_1: ┤1                                                      ├┤ X ├»
                 └───────────────────────────────────────────────────────┘└───┘»
 state_prep_anc: ──────────────────────────────────────────────────────────────»
                                           ┌───┐                               »
hhl_phase_reg_0: ──────────────────────────┤ H ├───────────────────────────────»
                                           ├───┤                               »
hhl_phase_reg_1: ──────────────────────────┤ H ├───────────────────────────────»
                                           ├───┤                               »
hhl_phase_reg_2: ──────────────────────────┤ H ├───────────────────────────────»
                                           ├───┤                               »
hhl_phase_reg_3: ──────────────────────────┤ H ├───────────────────────────────»
                                           ├───┤                               »
hhl_phase_reg_4: ──────────────────────────┤ H ├───────────────────────────────»
                                     ┌─────┴───┴─────┐                         »
        hhl_anc: ────────────────────┤ Ry(-0.043388) ├─────────────────────────»
                                     └───────────────┘                         »
«                      ┌───┐┌───┐┌───┐                                     »
«    gen_nodes_0: ──■──┤ X ├┤ H ├┤ H ├─────────────────────────────────────»
«                   │  └───┘└───┘└───┘┌───┐     ┌───┐           ┌───┐      »
«    gen_nodes_1: ──┼──────────────■──┤ X ├─────┤ H ├───────────┤ H ├──────»
«                   │  ┌───┐       │  └───┘┌────┴───┴─────┐┌────┴───┴─────┐»
«    tot_nodes_0: ──■──┤ X ├───────■───────┤0             ├┤0             ├»
«                   │  ├───┤┌───┐  │  ┌───┐│  circuit-101 ││  circuit-101 │»
«    tot_nodes_1: ──■──┤ X ├┤ X ├──■──┤ X ├┤1             ├┤1             ├»
«                 ┌─┴─┐└───┘└───┘┌─┴─┐├───┤└──────┬───────┘└──────┬───────┘»
« state_prep_anc: ┤ X ├──────────┤ X ├┤ X ├───────■───────────────■────────»
«                 └───┘          └───┘└───┘       │               │        »
«hhl_phase_reg_0: ────────────────────────────────┼───────────────┼────────»
«                                                 │               │        »
«hhl_phase_reg_1: ────────────────────────────────┼───────────────┼────────»
«                                                 │               │        »
«hhl_phase_reg_2: ────────────────────────────────┼───────────────┼────────»
«                                                 │               │        »
«hhl_phase_reg_3: ────────────────────────────────┼───────────────■────────»
«                                                 │                        »
«hhl_phase_reg_4: ────────────────────────────────■────────────────────────»
«                                                                          »
«        hhl_anc: ─────────────────────────────────────────────────────────»
«                                                                          »
«                                                                 »
«    gen_nodes_0: ────────────────────────────────────────────────»
«                                                                 »
«    gen_nodes_1: ────────────────────────────────────────────────»
«                 ┌──────────────┐┌──────────────┐┌──────────────┐»
«    tot_nodes_0: ┤0             ├┤0             ├┤0             ├»
«

In [19]:
# hhl_output_reg=ClassicalRegister(2, "hhl_output")
# hhl_circ[0].add_register(hhl_output_reg)
# # hhl_circ[0].measure(tot_nodes,hhl_output_reg)
backend=Aer.get_backend('aer_simulator')
hhl_circ[0].save_statevector()
hhl_circ[0]=transpile(hhl_circ[0], backend)   
result = backend.run(hhl_circ[0]).result()
statevector = result.get_statevector(hhl_circ[0])

In [20]:
hhl_output_reg=ClassicalRegister(2, "hhl_output")
hhl_circ[0].add_register(hhl_output_reg)
hhl_circ[0].measure(tot_nodes,hhl_output_reg)
hhl_circ[0]=transpile(hhl_circ[0], backend)   
result = backend.run(hhl_circ[0]).result()
# counts
# # {'00': 490, '11': 283, '10': 124, '01': 127}


In [21]:
list(statevector)[:2**4]

C:\Users\Dell\AppData\Local\Temp/ipykernel_20044/3448962520.py:1: DeprecationWarning: The return type of saved statevectors has been changed from a `numpy.ndarray` to a `qiskit.quantum_info.Statevector` as of qiskit-aer 0.10. Accessing numpy array attributes is deprecated and will result in an error in a future release. To continue using saved result objects as arrays you can explicitly cast them using  `np.asarray(object)`.
  list(statevector)[:2**4]


[(2.571780567419714e-33-4.6559355512136366e-46j),
 (4.200036401042746e-17-7.580984014313047e-30j),
 (4.2000364010427526e-17-7.476326754501596e-30j),
 (0.685917997575626-1.2319469437860386e-13j),
 (1.2858902837099495e-33-2.0673086229698242e-46j),
 (2.1000182005215547e-17-3.534432393935338e-30j),
 (2.1000182005215538e-17-3.5399137923834974e-30j),
 (0.3429589987878453-5.18633872628292e-14j),
 (-1.2858902837098542e-33+2.358887031241543e-46j),
 (-2.1000182005213572e-17+4.0969055850039735e-30j),
 (-2.100018200521368e-17+4.082742468516803e-30j),
 (-0.3429589987878187+6.504051088580612e-14j),
 (-1.9288354255648786e-33+3.1088628420680094e-46j),
 (-3.150027300782266e-17+4.945051278194763e-30j),
 (-3.150027300782252e-17+4.986246881438698e-30j),
 (-0.5144384981817477+7.863900559078633e-14j)]

In [28]:
list(statevector)[2**4:2**5]

C:\Users\Dell\AppData\Local\Temp/ipykernel_20044/247910180.py:1: DeprecationWarning: The return type of saved statevectors has been changed from a `numpy.ndarray` to a `qiskit.quantum_info.Statevector` as of qiskit-aer 0.10. Accessing numpy array attributes is deprecated and will result in an error in a future release. To continue using saved result objects as arrays you can explicitly cast them using  `np.asarray(object)`.
  list(statevector)[2**4:2**5]


[(3.3790957730396944e-65+1.574761419999384e-49j),
 (4.2581376167614545e-49+2.5717805674187068e-33j),
 (4.2581376167614545e-49+2.5717805674187068e-33j),
 (8.349459457071755e-33+4.20003640104093e-17j),
 (-2.4738555266675134e-63+7.873807099999389e-50j),
 (-3.969613617647423e-47+1.2858902837097627e-33j),
 (-3.969613617647423e-47+1.2858902837097627e-33j),
 (-6.6778168194615395e-31+2.100018200520992e-17j),
 (1.2634263839935088e-64-7.873807099998312e-50j),
 (-6.470094230558656e-48-1.2858902837095597e-33j),
 (-6.470094230558656e-48-1.2858902837095597e-33j),
 (-1.8608955760600598e-32-2.1000182005207172e-17j),
 (3.029382106635182e-63-1.181071065000117e-49j),
 (4.1408016884059767e-47-1.9288354255649877e-33j),
 (4.1408016884059767e-47-1.9288354255649877e-33j),
 (1.1374625177956864e-30-3.150027300782341e-17j)]

In [26]:
[statevector[3],
statevector[7],
statevector[11],
statevector[15]]

# array([ 0.5  ,  0.375, -0.75 , -0.875])
# [(1.1733252601426779e-15-7.093638722529953e-16j),
#  (2.351288456437307e-15-6.734635177308739e-16j),
#  (-0.24358325729500183+0.243583257295017j),
#  (-0.36537488594250805+0.3653748859425251j)]

[(0.685917997575626-1.2319469437860386e-13j),
 (0.3429589987878453-5.18633872628292e-14j),
 (-0.3429589987878187+6.504051088580612e-14j),
 (-0.5144384981817477+7.863900559078633e-14j)]

In [24]:
A_inv=np.linalg.inv(grid.A)
A_inv @ real_powers[:,0]

array([ 0.5  ,  0.375, -0.75 , -0.875])

In [25]:
real_powers[:,0]

array([ 2. ,  1. , -1. , -1.5])